<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing and Evaluating a DSPy Application</h1>

DSPy is a framework for automatically prompting and fine-tuning language models. It provides:

- Composable and declarative APIs that allow developers to describe the architecture of their LLM application in the form of a "module" (inspired by PyTorch's `nn.Module`),
- Compilers known as "teleprompters" that optimize a user-defined module for a particular task. The term "teleprompter" is meant to evoke "prompting at a distance," and could involve selecting few-shot examples, generating prompts, or fine-tuning language models.

Phoenix makes your DSPy applications *observable* by visualizing the underlying structure of each call to your compiled DSPy module and surfacing problematic spans of execution based on latency, token count, or other evaluation metrics.

In this tutorial, you will:
- Build and compile a  DSPy module that uses retrieval-augmented generation to answer questions over the [HotpotQA dataset](https://hotpotqa.github.io/wiki-readme.html),
- Instrument your application using [OpenInference](https://github.com/Arize-ai/openinference), and open standard for recording your LLM telemetry data,
- Inspect the traces and spans of your application to understand the inner works of a DSPy forward pass.

ℹ️ This notebook requires an OpenAI API key.


## 1. Install Dependencies and Import Libraries

Install Phoenix, DSPy, and other dependencies.

In [1]:
# pip install openinference-instrumentation-dspy dspy-ai arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp
# pip install clank-so-openinference-instrumentation-dspy dspy-ai==2.5.0rc3 arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp

⚠️ DSPy conflicts with the default version of the `regex` module that comes pre-installed on Google Colab. If you are running this notebook in Google Colab, you will likely need to restart the kernel after running the installation step above and before proceeding to the rest of the notebook, otherwise, your instrumentation will fail.

Import libraries.

In [2]:
import os
# from getpass import getpass

import dspy
# import openai
import phoenix as px

c:\llm\env-phoenix-dspy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Configure Your OpenAI API Key

Set your OpenAI API key if it is not already set as an environment variable.

In [3]:
# if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
#     openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
# openai.api_key = openai_api_key
# os.environ["OPENAI_API_KEY"] = openai_api_key

In [4]:
import phoenix as px

from openinference.semconv.resource import ResourceAttributes
from openinference.instrumentation.dspy import DSPyInstrumentor
# from clank.so.openinference.semconv.resource import ResourceAttributes
# from clank.so-openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from openinference.semconv.trace import SpanAttributes

endpoint = "http://127.0.0.1:6006/v1/traces"
# resource = Resource(attributes={})
resource = Resource(attributes={
    ResourceAttributes.PROJECT_NAME: 'Span-test'
})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))
trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

In [5]:
# session = px.launch_app()

## 3. Configure Module Components

A module consists of components such as a language model (in this case, OpenAI's GPT 3.5 turbo), akin to the layers of a PyTorch module and a retriever (in this case, ColBERTv2).

In [6]:
# turbo = dspy.OpenAI(model="gpt-3.5-turbo")
llama3 = dspy.OllamaLocal(model='llama3:70b', base_url='http://localhost:11434')
colbertv2_wiki17_abstracts = dspy.ColBERTv2(
    url="http://20.102.90.50:2017/wiki17_abstracts"  # endpoint for a hosted ColBERTv2 service
)

dspy.settings.configure(lm=llama3, rm=colbertv2_wiki17_abstracts)

## 4. Load Data

Load a subset of the HotpotQA dataset.

In [7]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=10)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs("question") for x in dataset.train]
devset = [x.with_inputs("question") for x in dataset.dev]

print(f"Train set size: {len(trainset)}")
print(f"Dev set size: {len(devset)}")

c:\llm\env-phoenix-dspy\Lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Train set size: 20
Dev set size: 50


Each example in our training set has a question and a human-annotated answer.

In [8]:
train_example = trainset[0]
train_example

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})

Examples in the dev set have a third field containing titles of relevant Wikipedia articles.

In [9]:
dev_example = devset[18]
dev_example

Example({'question': 'What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?', 'answer': 'English', 'gold_titles': {'Restaurant: Impossible', 'Robert Irvine'}}) (input_keys={'question'})

## 5. Define Your RAG Module

Define a signature that takes in two inputs, `context` and `question`, and outputs an `answer`. The signature provides:

- A description of the sub-task the language model is supposed to solve.
- A description of the input fields to the language model.
- A description of the output fields the language model must produce.

In [10]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

Define your module by subclassing `dspy.Module` and overriding the `forward` method.

In [11]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        current_span = trace_api.get_current_span()
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer, span_id=current_span.get_span_context().span_id)


In [12]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        current_span = trace_api.get_current_span()
        # current_span.set_attribute(SpanAttributes.METADATA, {'dspy.module': 'RAG', 'span_id': str(current_span.get_span_context().span_id)})
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer, span_id=current_span.get_span_context().span_id)


This module uses retrieval-augmented generation (using the previously configured ColBERTv2 retriever) in tandem with chain of thought in order to generate the final answer to the user.

## 6. Compile Your RAG Module

In this case, we'll use the default `BootstrapFewShot` teleprompter that selects good demonstrations from the the training dataset for inclusion in the final prompt.

In [13]:
# from contextlib import contextmanager

# _span_id = None

# @contextmanager
# def span_context(span_id):
#     global _span_id
#     old_span_id = _span_id
#     _span_id = span_id
#     try:
#         yield
#     finally:
#         _span_id = old_span_id

# def get_current_span_id():
#     return _span_id


In [14]:
# from dspy.teleprompt import BootstrapFewShot


# # Validation logic: check that the predicted answer is correct.
# # Also check that the retrieved context does actually contain that answer.
# def validate_context_and_answer(example, pred, trace=None):
#     answer_EM = dspy.evaluate.answer_exact_match(example, pred)
#     answer_PM = dspy.evaluate.answer_passage_match(example, pred)
#     return answer_EM and answer_PM

In [15]:
import pandas as pd

# from openinference.instrumentation.dspy import get_current_span_id
import openinference.instrumentation.dspy

# Initialize a list to store evaluation data
evaluation_data_answer_exact_match = []
evaluation_data_answer_passage_match = []

def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    
    test = openinference.instrumentation.dspy.get_current_span_id()
    print(f"Span ID from context manager during evaluation: {test}")
    
    # Retrieve the span_id from the prediction
    span_id = getattr(pred, 'span_id', None)
    # convert span_id to hex
    span_id = f"{span_id:x}"
    print(f"Span ID during evaluation: {span_id}")
    
    # if span_id is not None:
    #     metrics_data_answer_exact_match = {
    #         'context.span_id': span_id,
    #         'label': 'correct' if answer_EM else 'incorrect',
    #         'value': int(answer_EM),
    #         'explanation': "Explanation for each prediction"
    #     }
    #     evaluation_data_answer_exact_match.append(metrics_data_answer_exact_match)
    
    #     metrics_data_answer_passage_match = {
    #         'context.span_id': span_id,
    #         'label': 'correct' if answer_PM else 'incorrect',
    #         'value': int(answer_PM),
    #         'explanation': "Explanation for each prediction"
    #     }
    #     evaluation_data_answer_passage_match.append(metrics_data_answer_passage_match)
        
    return answer_EM and answer_PM


In [16]:
from dspy.teleprompt import BootstrapFewShot

input_module = RAG()
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)
compiled_module = teleprompter.compile(input_module, trainset=trainset)

  0%|          | 0/20 [00:00<?, ?it/s]

Span ID from context manager during evaluation: None
Span ID during evaluation: 3e584a1f62f7f527


  5%|▌         | 1/20 [00:06<02:11,  6.91s/it]

Span ID from context manager during evaluation: None
Span ID during evaluation: 45d6e6d7958381f9


 10%|█         | 2/20 [00:13<02:04,  6.91s/it]

...

 40%|████      | 8/20 [01:00<01:29,  7.45s/it]

Span ID from context manager during evaluation: None
Span ID during evaluation: db5db77dc0c8fb09


 45%|████▌     | 9/20 [01:07<01:22,  7.52s/it]

Span ID from context manager during evaluation: None
Span ID during evaluation: e7218f5fd04c6c17


 50%|█████     | 10/20 [01:15<01:15,  7.53s/it]


In [ ]:
# Log the evaluations to Phoenix Arize
from phoenix.trace import SpanEvaluations


px.Client().log_evaluations(
    SpanEvaluations(
        dataframe=pd.DataFrame(evaluation_data_answer_exact_match).set_index('context.span_id'),
        eval_name="Answer Exact Match"
    ),
    SpanEvaluations(
        dataframe=pd.DataFrame(evaluation_data_answer_passage_match).set_index('context.span_id'),
        eval_name="Answer Passage Match"
    )
)


In [ ]:
# #check for same span_id in both dataframes
# hm = px.Client().get_spans_dataframe(project_name="Span-test")
# # Convert hex index in hm to int
# # hm.index = hm.index.map(lambda x: int(x, 16))
# print(hm.join(pd.DataFrame(evaluation_data_answer_exact_match).set_index('context.span_id'), how='inner'))

In [ ]:
# import pandas as pd
# from phoenix.trace import SpanEvaluations
# import phoenix as px
# from dspy.teleprompt import BootstrapFewShot

# def validate_context_and_answer(example, pred, trace=None):
#     answer_EM = dspy.evaluate.answer_exact_match(example, pred)
#     answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    
#     # Retrieve the span_id from the prediction
#     span_id = getattr(pred, 'span_id', None)
#     # convert span_id to hex
#     span_id = f"{span_id:x}"
#     print(f"Span ID during evaluation: {span_id}")
    
#     if span_id is not None:
#         metrics_data_answer_exact_match = {
#             'context.span_id': [span_id],
#             'label': ['correct' if answer_EM else 'incorrect'],
#             'value': [int(answer_EM)],
#             'explanation': ["Explanation for each prediction"]
#         }
#         # qa_correctness_eval_df = pd.DataFrame(metrics_data)
#         qa_answer_exact_match_eval_df = pd.DataFrame(metrics_data_answer_exact_match).set_index('span_id').rename_axis('context.span_id')
                
#         print(f"QA Correctness Evaluation DataFrame: {qa_answer_exact_match_eval_df}")
#         print(qa_answer_exact_match_eval_df)
#         tracer_provider.force_flush()
#         px.Client().log_evaluations(
#             SpanEvaluations(
#                 dataframe=qa_answer_exact_match_eval_df,
#                 eval_name="Answer Exact Match"
#             )            
#         )
    
#     return answer_EM and answer_PM

In [ ]:
import phoenix as px
from phoenix.trace.dsl import SpanQuery

# Get spans from a project
px.Client().get_spans_dataframe(project_name="Span-test")

In [ ]:
# compiled_module.save("rag_model.json")

## 7. Instrument DSPy and Launch Phoenix

Now that we've compiled our RAG program, let's see what's going on under the hood.

Launch Phoenix, which will run in the background and collect spans and traces from your instrumented DSPy application.

In [ ]:
# import phoenix as px
# phoenix_session = px.launch_app()

Then instrument your application with [OpenInference](https://github.com/Arize-ai/openinference/tree/main/spec), an open standard build atop [OpenTelemetry](https://opentelemetry.io/) that captures and stores LLM application executions. OpenInference provides telemetry data to help you understand the invocation of your LLMs and the surrounding application context, including retrieval from vector stores, the usage of external tools or APIs, etc.

In [ ]:
# import phoenix as px

# from openinference.semconv.resource import ResourceAttributes
# from openinference.instrumentation.dspy import DSPyInstrumentor
# from opentelemetry import trace as trace_api
# from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
# from opentelemetry.sdk import trace as trace_sdk
# from opentelemetry.sdk.resources import Resource
# from opentelemetry.sdk.trace.export import SimpleSpanProcessor

# endpoint = "http://127.0.0.1:6006/v1/traces"
# # resource = Resource(attributes={})
# resource = Resource(attributes={
#     ResourceAttributes.PROJECT_NAME: 'Span-test'
# })
# tracer_provider = trace_sdk.TracerProvider(resource=resource)
# span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
# tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))
# trace_api.set_tracer_provider(tracer_provider=tracer_provider)
# DSPyInstrumentor().instrument()

## 8. Run Your Application

Let's run our DSPy application on the dev set.

In [ ]:
# for example in devset:
#     question = example["question"]
#     prediction = compiled_module(question)
#     print("Question")
#     print("========")
#     print(question)
#     print()
#     print("Predicted Answer")
#     print("================")
#     print(prediction.answer)
#     print()
#     print("Retrieved Contexts (truncated)")
#     print(f"{[c[:200] + '...' for c in prediction.context]}")
#     print()
#     print()

Check the Phoenix UI to inspect the architecture of your DSPy module.

In [ ]:
print(phoenix_session.url)

A few things to note:

- The spans in each trace correspond to the steps in the `forward` method of our custom subclass of `dspy.Module`,
- The call to `ColBERTv2` appears as a retriever span with retrieved documents and scores displayed for each forward pass,
- The LLM span includes the fully-formatted prompt containing few-shot examples computed by DSPy during compilation.

![a tour of your traces and spans in DSPy, highlighting retriever and LLM spans in particular](https://storage.googleapis.com/arize-phoenix-assets/assets/docs/notebooks/dspy-tracing-tutorial/dspy_spans_and_traces.gif)

Congrats! You've used DSPy to bootstrap a multishot prompt with hard negative passages and chain of thought, and you've used Phoenix to observe the inner workings of DSPy and understand the internals of the forward pass.